<a href="https://colab.research.google.com/github/GitMarco27/TMML/blob/main/Notebooks/012_pythonflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3 Minutes Machine Learning
## Episode 12: Pythonflow: from eager to graph python programming

#### Marco Sanguineti, 2021
---
Welcome to 3 minutes Machine Learning!

In [1]:
! pip install pythonflow -U -q

In [2]:
import tensorflow as tf
import pythonflow as pf
import numpy as np

# TensorflowGraph

In [3]:
class AddLayerOperation(pf.Operation):
    def __init__(self, model,  type_layers, num_nodes, activation):
        super(AddLayerOperation, self).__init__(model, type_layers, num_nodes, activation)


    def _evaluate(self, model, type_layers, num_nodes, activation):
        if type_layers=='Dense':
          new_layer = tf.keras.layers.Dense(num_nodes, activation=activation)

        model.add(new_layer)
        return model

class CompileOperation(pf.Operation):
    def __init__(self, model, number_name):
        super(CompileOperation, self).__init__(model, name='CompileOperation'+number_name)

    def _evaluate(self, model):
        model.compile(optimizer='adam', loss='mse')
        return model

class TrainOperation(pf.Operation):
    def __init__(self, model, epochs, batch_size, X, y):
        super(TrainOperation, self).__init__(model, epochs, batch_size,  X, y, name='TrainModel')

    def _evaluate(self, model,X, y,  epochs,batch_size):
        history = model.fit(X, y, epochs=epochs, batch_size=batch_size)
        return model, history

with pf.Graph() as graphadd:
    a = pf.constant(4)
    b = pf.constant(38)
    x = (a + b).set_name('x')

with pf.Graph() as graph:
    num_nodes = pf.placeholder(name="num_nodes")
    activation = pf.placeholder(name="activation")
    type_layers = pf.placeholder(name="type_layers")
    epochs = pf.placeholder(name="epochs")
    batch_size = pf.placeholder(name="batch_size")

    input_layer = pf.func_op(tf.keras.Input, [graphadd('x')])
    model = pf.func_op(tf.keras.models.Sequential, [input_layer], name='model')

    model_graph = AddLayerOperation(model, type_layers, num_nodes, activation)
    model_graph1 = AddLayerOperation(model_graph, type_layers, num_nodes, activation)
    model_graph2 = AddLayerOperation(model_graph1, type_layers, num_nodes, activation)
    model_graph_compile1 = CompileOperation(model_graph2, number_name='1')

    X = pf.func_op(np.random.rand, *[100, 42])
    y = pf.func_op(np.random.rand, *(100, 10))

    model_trained = TrainOperation(model_graph_compile1, X, y, epochs, batch_size)


model = graph(model_graph_compile1 , num_nodes=10, activation='relu', type_layers='Dense', 
               batch_size= 16, epochs=5)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                430       
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
 dense_2 (Dense)             (None, 10)                110       
                                                                 
Total params: 650
Trainable params: 650
Non-trainable params: 0
_________________________________________________________________


In [4]:
model, history = graph(model_trained , num_nodes=10, activation='relu', type_layers='Dense',
                       batch_size= 16, epochs=5)

Epoch 1/5
7/7 [==============================] - 1s 3ms/step - loss: 0.2547
Epoch 2/5
7/7 [==============================] - 0s 2ms/step - loss: 0.2324
Epoch 3/5
7/7 [==============================] - 0s 2ms/step - loss: 0.2085
Epoch 4/5
7/7 [==============================] - 0s 3ms/step - loss: 0.1864
Epoch 5/5
7/7 [==============================] - 0s 3ms/step - loss: 0.1689


In [5]:
import time
def crete_graph(num_nodes=10, activation='relu', type_layers='Dense',batch_size= 16, epochs=5 ): 
  models = graph(model_trained, num_nodes=num_nodes, activation=activation, type_layers=type_layers, 
                 batch_size= batch_size, epochs=epochs)
  return models

In [6]:
import pickle 

with open("ciao", 'wb') as f:
  pickle.dump(graph, f)

In [7]:
import pickle 

with open("ciao", 'rb') as f:
  new_graph = pickle.load(f)

del model
model, history = new_graph('TrainModel', num_nodes=10, activation='selu', type_layers='Dense',
                           batch_size= 16, epochs=5)
model.summary()


Epoch 1/5
7/7 [==============================] - 0s 2ms/step - loss: 0.6942
Epoch 2/5
7/7 [==============================] - 0s 3ms/step - loss: 0.5277
Epoch 3/5
7/7 [==============================] - 0s 2ms/step - loss: 0.4164
Epoch 4/5
7/7 [==============================] - 0s 3ms/step - loss: 0.3284
Epoch 5/5
7/7 [==============================] - 0s 2ms/step - loss: 0.2756
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 10)                430       
                                                                 
 dense_7 (Dense)             (None, 10)                110       
                                                                 
 dense_8 (Dense)             (None, 10)                110       
                                                                 
Total params: 650
Trainable params: 650
Non-trainable params: 0
______

In [8]:
vars(new_graph).keys()

dict_keys(['operations', 'dependencies'])

In [9]:
print(new_graph.operations)
print(new_graph.dependencies)

{'num_nodes': <pf.placeholder 'num_nodes'>, 'activation': <pf.placeholder 'activation'>, 'type_layers': <pf.placeholder 'type_layers'>, 'epochs': <pf.placeholder 'epochs'>, 'batch_size': <pf.placeholder 'batch_size'>, '46065a37df4b4915a5eeb90fb8960ecd': <pf.func_op '46065a37df4b4915a5eeb90fb8960ecd' target=<function Input at 0x7f2ad5afc9e0> args=<1 items> kwargs=<0 items>>, 'model': <pf.func_op 'model' target=<class 'keras.engine.sequential.Sequential'> args=<1 items> kwargs=<0 items>>, 'd182e9f85ff647d4893d12df272beb15': <__main__.AddLayerOperation object at 0x7f2ac92a7810>, '804045c6ee5146a68d1a9ed82fe61171': <__main__.AddLayerOperation object at 0x7f2ac92a78d0>, '67dbb624568841f4a49431d90650ea66': <__main__.AddLayerOperation object at 0x7f2ac92a7990>, 'CompileOperation1': <__main__.CompileOperation object at 0x7f2ac92a7a50>, 'a2e49d53fc7f47478471e540317166ab': <pf.func_op 'a2e49d53fc7f47478471e540317166ab' target=<built-in method rand of numpy.random.mtrand.RandomState object at 0x7

In [10]:
import threading
import time
import sys
import trace
import threading
import time
class thread_with_trace(threading.Thread):
  def __init__(self, *args, **keywords):
    threading.Thread.__init__(self, *args, **keywords)
    self.killed = False
 
  def start(self):
    self.__run_backup = self.run
    self.run = self.__run     
    threading.Thread.start(self)
 
  def __run(self):
    sys.settrace(self.globaltrace)
    self.__run_backup()
    self.run = self.__run_backup
 
  def globaltrace(self, frame, event, arg):
    if event == 'call':
      return self.localtrace
    else:
      return None
 
  def localtrace(self, frame, event, arg):
    if self.killed:
      if event == 'line':
        raise SystemExit()
    return self.localtrace
 
  def kill(self):
    self.killed = True

print(threading.active_count())
th = thread_with_trace(target=crete_graph, args=[10,'relu', 'Dense', 5, 16, 10])
th.start()
print(threading.active_count())
time.sleep(10)
th.kill()
th.join()
print('\n', threading.active_count())

10
11



PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "<ipython-input-10-75e4538b2b82>", line 18, in __run
    sys.settrace(self.globaltrace)

Exception in thread Thread-11:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "<ipython-input-10-75e4538b2b82>", line 19, in __run
    self.__run_backup()
  File "/usr/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
TypeError: crete_graph() takes from 0 to 5 positional arguments but 6 were given




 10
